In [42]:
import numpy as np
import pandas as pd
import h5py
import matplotlib.pyplot as plt
import neurokit2 as nk
import ast

In [ ]:
sample_freq = 500
num_patients = 15  # Первые 15 пациентов
plot_patient_idx = 0  # Индекс пациента для визуализации

metadata_full = pd.read_csv('metadata.csv')
metadata = metadata_full.head(num_patients).copy()

required_columns = ['ECG_ID', 'AHA_Code', 'Patient_ID', 'Age', 'Sex', 'N', 'Date']
if not all(col in metadata.columns for col in required_columns):
    raise ValueError("Файл metadata.csv не содержит всех необходимых столбцов.")

metadata['Age'] = metadata['Age'].astype(np.int32)
metadata['N'] = metadata['N'].astype(np.int32)

ecg_ids = metadata['ECG_ID'].values

ecg_signals = []
for ecg_id in ecg_ids:
    with h5py.File(f'records/{ecg_id}.h5', 'r') as f:
        ecg_signals.append(f['ecg'][()])

In [ ]:
def process_ecg(signal, sampling_rate):
    try:
        cleaned_signal = nk.ecg_clean(signal, sampling_rate=sampling_rate, method="biosppy")
        
        # Детекция R-пиков
        peaks_result = nk.ecg_peaks(cleaned_signal, sampling_rate=sampling_rate, method="pantompkins")

        if isinstance(peaks_result, tuple):
            peaks_info = None
            for item in peaks_result:
                if isinstance(item, dict) and "ECG_R_Peaks" in item:
                    peaks_info = item
                    break
            if peaks_info is None:
                print("Не удалось найти словарь с данными пиков.")
                return None, None
        else:
            peaks_info = peaks_result

        binary_r_peaks = peaks_info.get("ECG_R_Peaks", [])
        r_peaks = np.where(binary_r_peaks)[0]
        
        if len(r_peaks) == 0:
            print("Массив R-пиков пуст. Пропускаем delineate для данного сигнала.")
            return peaks_info, {}
        
        # Детекции границ волн
        waves = {}
        # 1. Пробуем метод "dwt"
        try:
            delineation = nk.ecg_delineate(
                cleaned_signal, 
                rpeaks=r_peaks,
                sampling_rate=sampling_rate, 
                method="dwt"
            )
        except ValueError as e:
            if "zero-size array" in str(e):
                print("Ошибка: np.min пустого массива внутри nk.ecg_delineate с методом dwt. Пробуем метод peak.")
                try:
                    delineation = nk.ecg_delineate(
                        cleaned_signal, 
                        rpeaks=r_peaks,
                        sampling_rate=sampling_rate, 
                        method="peak"
                    )
                except ValueError as e2:
                    if "zero-size array" in str(e2):
                        print("Ошибка при delineate методом peak: ", e2)
                        print("Пробуем метод cwt.")
                        # 2. Пробуем метод "сwt"
                        try:
                            delineation = nk.ecg_delineate(
                                cleaned_signal, 
                                rpeaks=r_peaks,
                                sampling_rate=sampling_rate, 
                                method="cwt"
                            )
                        except Exception as e3:
                            print("Ошибка при delineate методом cwt: ", e3)
                            delineation = None
                    else:
                        raise e2
            else:
                raise e

        if delineation is not None:
            if isinstance(delineation, tuple):
                for item in delineation:
                    if isinstance(item, dict) and "ECG_P_Onsets" in item:
                        waves = item
                        break
            elif isinstance(delineation, dict):
                waves = delineation

        if waves is None:
            waves = {}
        
        return peaks_info, waves
    except Exception as e:
        print(f"Ошибка при обработке сигнала: {e}")
        return None, None

In [ ]:
def create_annotations(peaks_info, waves):
    annotations = []
    if peaks_info is None or waves is None:
        return annotations

    r_peaks = peaks_info.get("ECG_R_Peaks", [])

    for i in range(1, len(r_peaks)):
        cycle = {
            'P_start': waves.get('ECG_P_Onsets', [])[i-1] if i-1 < len(waves.get('ECG_P_Onsets', [])) else np.nan,
            'P_end': waves.get('ECG_P_Offsets', [])[i-1] if i-1 < len(waves.get('ECG_P_Offsets', [])) else np.nan,
            'Q_start': waves.get('ECG_Q_Onsets', [])[i-1] if i-1 < len(waves.get('ECG_Q_Onsets', [])) else np.nan,
            'Q_max': r_peaks[i-1],
            'R_max': r_peaks[i],
            'S_max': waves.get('ECG_S_Peaks', [])[i-1] if i-1 < len(waves.get('ECG_S_Peaks', [])) else np.nan,
            'S_end': waves.get('ECG_S_Offsets', [])[i-1] if i-1 < len(waves.get('ECG_S_Offsets', [])) else np.nan,
            'T_start': waves.get('ECG_T_Onsets', [])[i-1] if i-1 < len(waves.get('ECG_T_Onsets', [])) else np.nan,
            'T_end': waves.get('ECG_T_Offsets', [])[i-1] if i-1 < len(waves.get('ECG_T_Offsets', [])) else np.nan
        }
        annotations.append(cycle)
    return annotations

In [53]:
for patient_idx, (ecg_id, signal) in enumerate(zip(ecg_ids, ecg_signals)):
    if signal is None:
        print(f"Сигнал для пациента {ecg_id} отсутствует, пропускаем.")
        continue

    patient_annotations = []

    if signal.ndim == 1:
        signal = signal.reshape(1, -1)

    for channel in range(signal.shape[0]):
        channel_signal = signal[channel, :]
        try:
            peaks_info, waves = process_ecg(channel_signal, sample_freq)
            annotations = create_annotations(peaks_info, waves)
            patient_annotations.extend(annotations)
        except Exception as e:
            print(f"Ошибка при обработке пациента {ecg_id}, канал {channel}: {e}")

    df_patient = pd.DataFrame(patient_annotations)
    df_patient.to_csv(f'patient_{ecg_id}_annotations.csv', index=False)
    print(f"Файл для пациента {ecg_id} сохранен.")

Файл для пациента A00001 сохранен.
Файл для пациента A00002 сохранен.
Файл для пациента A00003 сохранен.
Файл для пациента A00004 сохранен.
Файл для пациента A00005 сохранен.
Файл для пациента A00006 сохранен.
Файл для пациента A00007 сохранен.
Файл для пациента A00008 сохранен.
Файл для пациента A00009 сохранен.
Файл для пациента A00010 сохранен.
Файл для пациента A00011 сохранен.
Файл для пациента A00012 сохранен.
Файл для пациента A00013 сохранен.
Файл для пациента A00014 сохранен.
Файл для пациента A00015 сохранен.
